In [5]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:



import pandas as pd
import numpy as np
import pymysql 


# In[2]:



def connections():
    db = pymysql.connect(host='HA-Proxy-SQL-Mafiree-2129065769.ap-south-1.elb.amazonaws.com' , port=6033 , user='reportsud' , passwd='s1r1e2p3o4r5!!', db='ecomm')
    return db
def value_interpreter(number,limit):
    val = int(number/limit)
    if (number%limit)==0:
        val_add = 0
    else:
        val_add = 1
        
    final_val = val+val_add
    list_values = [0]
    for i in range(final_val):
        value_append = (i+1)*limit
        if value_append<number:
            list_values.append((i+1)*limit)
        else:
            list_values.append(number)
    return final_val,list_values


# In[ ]:


input_data = pd.read_csv('forward_awb1.csv')


# In[ ]:



print('AWB Input Value: %d'%input_data[input_data.columns[0]].count())


# In[3]:



#Getting_raw_details
query_raw_data_start = '''
select sh.airwaybill_number 'awb',sh.declared_value,
case when sh.reverse_pickup= 1 then 'REVERSE' WHEN sh.rts_status= 1 then'RTS' else  'FORWARD' END 'Shipment_type',
@date_consi:= sh.added_on 'Added_date',
#case when (sh.inscan_date>sh.added_on) then sh.added_on else sh.inscan_date end 'Added_date',
concat('service_centre_shipmenthistory_',year(@date_consi),'_',case when month(@date_consi)<10 then concat('0',month(@date_consi)) else
month(@date_consi) end) 'Applicable_History_Table'
from service_centre_shipment sh
where sh.airwaybill_number in 
(
'''
query_raw_data_end = ')'


# In[6]:


try:
    del data
except:
    pass

try:
    conn = connections()
    print('Working in 50 K batches\nTotal AWB: {}'.format(len(input_data[input_data.columns[0]])))
    data_list_1 = []
    a,b = value_interpreter(len(input_data[input_data.columns[0]]),50000)
    for i,j in enumerate(b):
        if ((i+1)==len(b)):
            pass
        else:
            print('Working Range : ',str(j),'-',str(b[i+1]))

            query_data = query_raw_data_start+(',').join(input_data[input_data.columns[0]][j:b[i+1]].apply(lambda x:str(int(x))))+query_raw_data_end
            conn = connections()
            temp_data = pd.read_sql(query_data,conn)
            print('Saving_data of Batch: %d of Total Batch %d\n'%(i+1,len(b)-1))
            data_list_1.append(temp_data)
            #temp_data.to_csv(fd_path+str(i)+'_fd_1_.csv',index=False)
    for i in data_list_1:
        try:
            data = data.append(i)
        except:
            data = i

    print('Raw Data imported from SQL')
    data = data.reset_index().drop(columns='index')
    print('\nBiforcation')
    print(data['Shipment_type'].value_counts())
    print('\nHistory Table Applicable')
    print(data['Applicable_History_Table'].value_counts())
except:
    print('Error in phase 1')


data.to_csv('required_data.csv')

def query_history(query_start,query_end,history_table,word ='xxxx_xxxx_xxxx_xxxx' ):
    return query_start.replace(word,history_table),query_end.replace(word,history_table)


#r_sql_script

# select 
#             sh.airwaybill_number as 'Airwaybill no',
#             sh.order_number as 'Order no',
#             sh.product_type as 'Product Type',
#             sh.declared_value as 'Declared value',
#             sh.collectable_value as 'Collectable Value',
#             ' ' as 'Pickup Type',
#             IFNULL(DATE_FORMAT(sh.inscan_date,'%d-%b-%Y %H:%i:%s'),'') as 'Pickup Date',
#             case when sh.rts_status IN (1) then ls.pincode when sh.reverse_pickup=1 then la.pincode else sh.pincode end as 'Destination Pincode',
#             case when sh.reverse_pickup = 1 then rs.pickup_pincode when sh.rts_status = 1 then sh.pincode else p.pincode end as 'Origin Pincode',
#             IFNULL(sc.center_name,sc1.center_name) as 'Origin SC',
#             case when sh.rts_status IN (1) then l_dst.center_name else l_dst.center_name end as 'Destination SC',
#             dest_orig.center_name as 'Manifest Location',
#             r_org.region_name as 'Origin Region',
#             r_dst.region_name as 'Destn Region',
#             s_org.state_name as 'Origin State',
#             s_dst.state_name as 'Destn State',
#             CONCAT(c.name,' - ' , c.code) as 'Shipper',
#             IFNULL(DATE_FORMAT(sh.added_on, '%d-%b-%Y %H:%i'),'') as 'Registered Time',
#             ifnull((select distinct 'Yes' from addon_services_shipmentqualitycheck where shipment_id=sh.id),'No') as 'RVP QC?',
#             ifnull((select group_concat(distinct upper(aos.code) separator ',') from  addon_services_addonservices aos inner join addon_services_shipmentservices aoss on aoss.addonservice_id = aos.id where aoss.shipment_id=sh.id),'N/A') as 'VAS Services',
#             sh.pieces as 'No. Of Pieces',
#             sh.rts_status as 'RTS Status',
#             CASE WHEN (IFNULL(api.transport_type,0) = 0) THEN 'No' ELSE 'Yes' END as 'DG Shipment',
#             CASE WHEN (sh.original_dest_id <> sh.service_centre_id and sh.rts_status<>1) THEN concat('Yes (', dest_orig.center_shortcode , ' -> ', l_dst.center_shortcode, ')' ) ELSE 'No' END as 'Misrouted',
#             IFNULL((select LEFT(concat(group_concat(case when (rto.status=0) then concat('Yes (', DATE_FORMAT(rto.updated_on,'%d-%b-%Y %H:%i'),')') when (status=1) then concat('Revoked (', DATE_FORMAT(rto.updated_on,'%d-%b-%Y %H:%i'),')') else '' end order by updated_on desc separator '|'),'|'), INSTR(concat(group_concat(case when (rto.status=0) then concat('Yes (', DATE_FORMAT(rto.updated_on,'%d-%b-%Y %H:%i'),')') when (status=1) then concat('Revoked (', DATE_FORMAT(rto.updated_on,'%d-%b-%Y %H:%i'),')') else '' end order by updated_on desc separator '|'),'|'),'|')-1) latest_status  from apiv2_shipmentcancelqueue rto where rto.airwaybill_number = sh.airwaybill_number group by airwaybill_number),'') as 'RTO Lock Status',
#             IFNULL((select LEFT(concat(group_concat(case when (ofd.status=0) then concat('Yes (', DATE_FORMAT(ofd.updated_on,'%d-%b-%Y %H:%i'),')') when (ofd.status=1) then concat('Revoked (', DATE_FORMAT(ofd.updated_on,'%d-%b-%Y %H:%i'),')') else '' end order by ofd.updated_on desc separator '|'),'|'), INSTR(concat(group_concat(case when (ofd.status=0) then concat('Yes (', DATE_FORMAT(ofd.updated_on,'%d-%b-%Y %H:%i'),')') when (ofd.status=1) then concat('Revoked (', DATE_FORMAT(ofd.updated_on,'%d-%b-%Y %H:%i'),')') else '' end order by ofd.updated_on desc separator '|'),'|'),'|')-1) latest_status from service_centre_ofdlockqueue ofd where ofd.shipment_id = sh.id group by ofd.shipment_id),'') as 'OFD Lock Status',
#             sh.updated_on as 'Status Updated On',
#             COALESCE((SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM ",his_tab_1," h WHERE h.shipment_id = sh.id AND  h.status IN (50,4,5,2011,2001,2005,2008) order by id  limit 1),(SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM ",his_tab_2," h WHERE h.shipment_id = sh.id AND  h.status IN (50,4,5,2011,2001,2005,2008) order by id  limit 1),(SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM ",his_tab_3," h WHERE h.shipment_id = sh.id AND  h.status IN (50,4,5,2011,2001,2005,2008) order by id  limit 1), null) as 'First hub inscan time',
#             COALESCE((SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM ",his_tab_1," h WHERE h.shipment_id = sh.id AND  h.status IN (26,2000) order by id  limit 1),(SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM ",his_tab_2," h WHERE h.shipment_id = sh.id AND  h.status IN (26,2000) order by id  limit 1),(SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM ",his_tab_3," h WHERE h.shipment_id = sh.id AND  h.status IN (26,2000) order by id  limit 1),null) as 'First hub outscan time',
#             COALESCE((SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM ",his_tab_1," h WHERE h.shipment_id = sh.id AND  h.status IN (50,4,5,2011,2001,2005,2008) order by id desc limit 1),(SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM ",his_tab_2," h WHERE h.shipment_id = sh.id AND  h.status IN (50,4,5,2011,2001,2005,2008) order by id desc limit 1),(SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM ",his_tab_3," h WHERE h.shipment_id = sh.id AND  h.status IN (50,4,5,2011,2001,2005,2008) order by id desc limit 1),null) as 'Latest hub inscan time',
#             COALESCE((SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM ",his_tab_1," h WHERE h.shipment_id = sh.id AND  h.status IN (26,2000) order by id desc limit 1),(SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM ",his_tab_2," h WHERE h.shipment_id = sh.id AND  h.status IN (26,2000) order by id desc limit 1),(SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM ",his_tab_3," h WHERE h.shipment_id = sh.id AND  h.status IN (26,2000) order by id desc limit 1),null) as 'Latest hub outscan time',
#             IFNULL(DATE_FORMAT(ship_latest_status_update_time, '%d-%b-%Y %H:%i:%s'),'') as 'Last Scan',
#             IFNULL(DATE_FORMAT(ifnull(case when (sm.parent_code in (1224)) then IFNULL((select r.scheduled_delivery_date from track_me_callcentrecomment c inner join track_me_callcentrecommentresolution r on c.id = r.call_centre_comment_id where c.shipments_id = sh.id order by c.date desc limit 1 ),Null) else Null end, (select IFNULL(DATE_FORMAT(IFNULL(STR_TO_DATE(add_info_value,'%Y-%m-%d'),STR_TO_DATE(add_info_value,'%d/%m/%Y')),'%d-%b-%Y'),add_info_value)  from service_centre_additionalinformation where shipment_id = sh.id and add_info_key = 'DATE' LIMIT 1)), '%d-%b-%Y'),'') as 'Scheduled Date',
#             IFNULL(last_added_bag_number,'') as 'Bag Number',
            
#             @latest_status := CONCAT(IFNULL((select LEFT(concat(group_concat(status order by updated_on desc separator '|'),'|'), INSTR(concat(group_concat(status order by updated_on desc separator '|'),'|'),'|')-1) latest_status  from ",his_tab_1," where shipment_id=sh.id and status not in (11,12,16, 20, 75,88) group by shipment_id),''), IFNULL((select LEFT(concat(group_concat(status order by updated_on desc separator '|'),'|'), INSTR(concat(group_concat(status order by updated_on desc separator '|'),'|'),'|')-1) latest_status  from ",his_tab_2," where shipment_id=sh.id and status not in (11,12,16, 20, 75,88) group by shipment_id),''),IFNULL((select LEFT(concat(group_concat(status order by updated_on desc separator '|'),'|'), INSTR(concat(group_concat(status order by updated_on desc separator '|'),'|'),'|')-1) latest_status from ",his_tab_3," where shipment_id=sh.id and status not in (11,12,16, 20, 75,88) group by shipment_id),'')) as 'Status Number',
#             CONCAT( CASE WHEN (@latest_status=8 and sh.rts_status=2) THEN 'Returned' WHEN (IFNULL(ship_current_status_code,'') = '777' and sh.rts_status=2) THEN 'Returned' WHEN (@latest_status in (0,310)) THEN 'Shipment Uploaded' WHEN (@latest_status=1) THEN 'Pickup Complete / Inscan' WHEN (@latest_status=2) THEN 'Inscan completion / Ready for Bagging' WHEN (@latest_status=3) THEN 'Bagging completed' WHEN (@latest_status=4) THEN 'Shipment at HUB' WHEN (@latest_status=5) THEN 'Bagging completed at Hub' WHEN (@latest_status=6) THEN 'Shipment at Delivery Centre' WHEN (@latest_status=7) THEN 'Outscan' WHEN (@latest_status=8) THEN 'Undelivered' WHEN (@latest_status=9) THEN 'Delivered / Closed' WHEN (@latest_status=11) THEN 'Alternate Instruction given' WHEN (@latest_status=12) THEN 'Complaint Registered' WHEN (@latest_status=13) THEN 'Assigned to Run Code' WHEN (@latest_status=14) THEN 'Airport Confirmation Sucessfull, connected to destination via Service Centre' WHEN (@latest_status=15) THEN 'Airport Confirmation Successfull, connected to destination via Hub' WHEN (@latest_status=17) THEN CASE WHEN (IFNULL(sh.ref_airwaybill_number,'') = '') THEN 'Redirection under Same Airwaybill' ELSE concat('Redirection under new airwaybill number ' , IFNULL(sh.ref_airwaybill_number,'')) END WHEN (@latest_status=20) THEN 'Undelivered' WHEN (@latest_status=21) THEN  concat('Tele Calling (', ifnull(sm.code_description,''), ')') WHEN (@latest_status=26) THEN IFNULL(ship_remarks,'Connected from Hub') WHEN (@latest_status=27) THEN IFNULL(ship_remarks,'Connected from SC') WHEN (@latest_status=31) THEN 'Reverse registered' WHEN (@latest_status=32) THEN 'Reverse out for pickup' WHEN (@latest_status=33) THEN 'Reverse not picked up' WHEN (@latest_status=40) THEN 'Mass Update' WHEN (@latest_status=50) THEN 'Inscan At Hub' WHEN (@latest_status=55) THEN 'Bag Inscan At DC' WHEN (@latest_status=77) THEN 'Shipment RTO Lock' WHEN (@latest_status=80) THEN 'RTO Lock Revoked' WHEN (@latest_status=82) THEN 'OFD Lock Applied' WHEN (@latest_status=83) THEN 'OFD Lock Revoked' WHEN (ship_current_status_code=333) THEN 'Lost' WHEN (@latest_status=2000) THEN concat('In-Transit (',ifnull(ship_remarks,''),')') WHEN (@latest_status=2001) THEN 'Bag Debagged at HUB' WHEN (@latest_status=2002) THEN 'Bag Debagged at DC' WHEN (@latest_status=2003) THEN 'Bag Debagged at PPC' WHEN (@latest_status=2004) THEN 'Trip inscan at PPC' WHEN (@latest_status=2005) THEN 'Trip inscan at HUB' WHEN (@latest_status=2006) THEN 'Trip inscan at DC' WHEN (@latest_status=2007) THEN 'Trip Unload at PPC' WHEN (@latest_status=2008) THEN 'Trip Unload at HUB' WHEN (@latest_status=2009) THEN 'Trip Unload at DC' WHEN (@latest_status=2010) THEN 'Trip load / unload complete at PPC' WHEN (@latest_status=2011) THEN 'Trip load / unload complete at HUB' WHEN (@latest_status=2012) THEN 'Trip load / unload complete at DC' ELSE @latest_status END ) as 'Status',
#             @latest_reason_code := ifnull(sm.code,'') as 'Reason Code',
#             ifnull(CASE WHEN (@latest_reason_code='') THEN '' else (select IFNULL(code_description,'') from ecomm_admin_shipmentstatusmaster where code=@latest_reason_code) END,'') as 'Reason Code Description',
#             replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(CONCAT(IFNULL((select LEFT(concat(group_concat(remarks order by updated_on desc separator '|'),'|'), INSTR(concat(group_concat(remarks order by updated_on desc separator '|'),'|'),'|')-1) latest_status  from ",his_tab_3," where shipment_id=sh.id  group by shipment_id),''), IFNULL((select LEFT(concat(group_concat(remarks order by updated_on desc separator '|'),'|'), INSTR(concat(group_concat(remarks order by updated_on desc separator '|'),'|'),'|')-1) latest_status  from ",his_tab_2," where shipment_id=sh.id  group by shipment_id),''), IFNULL((select LEFT(concat(group_concat(remarks order by updated_on desc separator '|'),'|'), INSTR(concat(group_concat(remarks order by updated_on desc separator '|'),'|'),'|')-1) latest_status  from ",his_tab_1," where shipment_id=sh.id  group by shipment_id),'') ),',',''),'/n',''),char(9),''),char(10),''),char(13),''),char(34),''),1,'X'),2,'X'),3,'X'),4,'X'),5,'X'),6,'X'),7,'X'),8,'X'),9,'X'),0,'X') as 'Remarks',
#             (select count(distinct case when scdo.status > 0 and sh.product_type in ('eds') then scdo.deliveryoutscan_id when sh.rts_status in (0,2) and sh.product_type not in ('eds') then scdo.deliveryoutscan_id  else null end) from service_centre_doshipment scdo where sh.id = scdo.shipment_id limit 1) as  'No_of_attempts',
            
#             ifnull(COALESCE((SELECT lsc.center_shortcode FROM ",his_tab_1," h left join location_servicecenter lsc on lsc.id = h.current_sc_id  WHERE h.shipment_id = sh.id  AND h.status IN (1,2,3,4,6,50,55) order by h.id desc limit 1),(SELECT lsc.center_shortcode FROM ",his_tab_2," h left join location_servicecenter lsc on lsc.id = h.current_sc_id WHERE h.shipment_id = sh.id  AND h.status IN (1,2,3,4,6,50,55) order by h.id desc limit 1),(SELECT lsc.center_shortcode FROM ",his_tab_3," h left join location_servicecenter lsc on lsc.id = h.current_sc_id WHERE h.shipment_id = sh.id AND h.status IN (1,2,3,4,6,50,55) order by h.id desc limit 1), null),CASE WHEN (IFNULL(bag_latest_inscan_hub_time,'1900-01-01') > IFNULL(bag_latest_inscan_dc_time,'1900-01-01')) THEN latest_sc_bag1.center_name ELSE IFNULL(IFNULL(latest_sc1.center_name,latest_sc.center_name),'') END ) as 'Current SC',
#             COALESCE((SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM ",his_tab_1," h WHERE h.shipment_id = sh.id AND h.current_sc_id = sh.service_centre_id AND h.status IN (55,6) order by id limit 1),(SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM ",his_tab_2," h WHERE h.shipment_id = sh.id AND h.current_sc_id = sh.service_centre_id AND h.status IN (55 , 6) order by id limit 1),(SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM ",his_tab_3," h WHERE h.shipment_id = sh.id AND h.current_sc_id = sh.service_centre_id AND h.status IN (55 , 6) order by id limit 1), null) as 'Inscan at DC Date',
#             COALESCE((SELECT lsc.center_shortcode FROM ",his_tab_1," h left join location_servicecenter lsc on lsc.id = h.current_sc_id  WHERE h.shipment_id = sh.id AND h.current_sc_id = sh.service_centre_id AND h.status IN (55,6) order by h.id limit 1),(SELECT lsc.center_shortcode FROM ",his_tab_2," h left join location_servicecenter lsc on lsc.id = h.current_sc_id WHERE h.shipment_id = sh.id AND h.current_sc_id = sh.service_centre_id AND h.status IN (55 , 6) order by h.id limit 1),(SELECT lsc.center_shortcode FROM ",his_tab_3," h left join location_servicecenter lsc on lsc.id = h.current_sc_id WHERE h.shipment_id = sh.id AND h.current_sc_id = sh.service_centre_id AND h.status IN (55 , 6) order by h.id limit 1),null) as 'Inscanned DC',
            
#             (Select IFNULL(DATE_FORMAT(min(scdo.added_on),'%d-%b-%Y %H:%i:%s'),'') from  service_centre_doshipment scdo inner join service_centre_deliveryoutscan scde on scde.id = scdo.deliveryoutscan_id where sh.id = scdo.shipment_id and (scdo.added_on > sh.inscan_date )) as 'First Attempted Date',
#             CASE WHEN (IFNULL(ship_first_status_update_reason_code,'')='') THEN IFNULL(mass_updation_status,'') ELSE IFNULL(ship_first_status_update_reason_code,'') END as 'First Reason Code',
            
#             IFNULL(sm_first.code_description,IFNULL(mass_updation_status,'')) as 'First Reason',
            
#             CASE WHEN (IFNULL(ship_latest_status_update_reason_code,'')='') THEN IFNULL(mass_updation_status,'') ELSE IFNULL(ship_latest_status_update_reason_code,'') END as 'Last Reason Code',
#             IFNULL(sm_last.code_description,IFNULL(mass_updation_status,'')) as 'Last Reason',
#             IFNULL(sh.ref_airwaybill_number,'') as 'Ref AWB',
#             fwd.inscan_date as 'Parent Inscan Date',
            
#             fwd.added_on as 'Parent Added Date',
#             (Select IFNULL(DATE_FORMAT(max(scdo.added_on),'%d-%b-%Y %H:%i:%s'),'') from  service_centre_doshipment scdo inner join service_centre_deliveryoutscan scde on scde.id = scdo.deliveryoutscan_id where sh.id = scdo.shipment_id and (scdo.added_on > sh.inscan_date )) as 'Last Outscan time',
#             (Select IFNULL(DATE_FORMAT(min(scdo.added_on),'%d-%b-%Y %H:%i:%s'),'') from  service_centre_doshipment scdo inner join service_centre_deliveryoutscan scde on scde.id = scdo.deliveryoutscan_id where sh.id = scdo.shipment_id and sh.reverse_pickup = 1 and (scdo.added_on <= sh.inscan_date or sh.inscan_date is null)) as 'RVP_first_pickup_attempt',
#             (Select IFNULL(DATE_FORMAT(max(scdo.added_on),'%d-%b-%Y %H:%i:%s'),'') from  service_centre_doshipment scdo inner join service_centre_deliveryoutscan scde on scde.id = scdo.deliveryoutscan_id where sh.id = scdo.shipment_id and sh.reverse_pickup = 1 and (scdo.added_on <= sh.inscan_date or sh.inscan_date is null)) as 'RVP_latest_pickup_attempt',
            
#             CONCAT(IFNULL((select ifnull(DATE_FORMAT(max(h1.updated_on), '%d-%b-%Y %H:%i'),'') latest_status from ",his_tab_1," h1 where shipment_id=sh.id and status not in (11,12,16, 20, 75,77,80) and reason_code_id not in(127,84,85,86,87,88,77,116,117,228,229,230,232,232,116,117)),'') ,IFNULL((select ifnull(DATE_FORMAT(max(h1.updated_on), '%d-%b-%Y %H:%i'),'') latest_status from ",his_tab_2," h1 where shipment_id=sh.id and status not in (11,12,16, 20, 75,77,80) and reason_code_id not in(127,84,85,86,87,88,77,116,117,228,229,230,232,232,116,117)),'') ,IFNULL((select ifnull(DATE_FORMAT(max(h1.updated_on), '%d-%b-%Y %H:%i'),'') latest_status from ",his_tab_3," h1 where shipment_id=sh.id and status not in (11,12,16, 20, 75,77,80) and reason_code_id not in(127,84,85,86,87,88,77,116,117,228,229,230,232,232,116,117)),'')) as 'Last Updated date',
            
#             ifnull((select date(sa.add_info_value) from service_centre_additionalinformation sa where sh.id = sa.shipment_id and sa.add_info_key='ORIGINAL_EDD'  limit 1),'') as 'Actual_EDD',
            
#             date(sh.expected_dod) as 'Updated_EDD',
#             @latest_Outscanned_time := (select o.added_on from service_centre_deliveryoutscan o inner join service_centre_doshipment ds on o.id = ds.deliveryoutscan_id where ds.shipment_id = sh.id and o.status>0 order by o.added_on desc limit 1) as 'latest Outscanned time',(select concat(ifnull(ae.firstname,''),' ',ifnull(ae.lastname,''),'- (',ae.employee_code,')') from service_centre_deliveryoutscan o inner join service_centre_doshipment ds on o.id = ds.deliveryoutscan_id left join authentication_employeemaster ae on ae.id = o.employee_code_id where ds.shipment_id = sh.id and o.status>0 and o.added_on = @latest_Outscanned_time order by o.added_on desc limit 1) as 'latest Outscanned by'
            
            
#             from service_centre_shipment sh inner join customer_customer c on c.id = sh.shipper_id
##             left join reports_shipmentbaghistory s on sh.airwaybill_number=s.awb
#             left join service_centre_shipment fwd on fwd.airwaybill_number = sh.ref_airwaybill_number
#             left join location_servicecenter l_dst on sh.service_centre_id = l_dst.id
#             left join location_city c_dst on c_dst.id = l_dst.city_id
#             left join location_state s_dst on s_dst.id = c_dst.state_id
#             left join location_region r_dst on r_dst.id = c_dst.region_id
#             left join ecomm_admin_shipmentstatusmaster sm on sm.id = sh.reason_code_id
#             left join pickup_pickupregistration p on sh.pickup_id=p.id
#             left join location_servicecenter sc on p.service_centre_id = sc.id
#             left join location_city c_org on c_org.id = sc.city_id
#             left join location_state s_org on s_org.id = c_org.state_id
#             left join location_region r_org on r_org.id = c_org.region_id
#             left join location_servicecenter sc1 on s.ship_added_on_sc = sc1.center_shortcode 
#             left join ecomm_admin_shipmentstatusmaster sm_last on sm_last.code = ifnull(s.ship_latest_status_update_reason_code,mass_updation_status) 
##            left join location_servicecenter latest_sc1 on latest_sc1.id=sh.current_sc_id
#             left join location_servicecenter dest_orig on dest_orig.id=sh.original_dest_id
#             left join api_apishipments api on api.shipment_id = sh.id
##             left join location_servicecenter latest_sc_bag on latest_sc_bag.center_shortcode=bag_latest_inscan_dc_sc
##             left join location_servicecenter latest_sc_bag1 on latest_sc_bag1.center_shortcode=bag_latest_inscan_hub_sc
#             left join ecomm_admin_shipmentstatusmaster sm_first on sm_first.code = ifnull(s.ship_first_status_update_reason_code,mass_updation_status)
##             left join location_servicecenter latest_sc on latest_sc.center_shortcode=ship_latest_inscan_dc_sc
#             left join customer_shipper cs on cs.id = p.return_subcustomer_code_id
#             left join location_address ls on ls.id = cs.address_id
#             left join location_pincode rp on rp.pincode = ls.pincode
#             left join location_servicecenter rc on rc.id =rp.service_center_id
#             left join service_centre_reverseshipment rs on sh.airwaybill_number = rs.airwaybill_number
#             left join pickup_pickupregistration p1 on p1.id = rs.pickup_id
#             left join customer_shipper cs1 on cs1.id = p1.subcustomer_code_id
#             left join location_address la on la.id = cs1.address_id
            
#             where sh.airwaybill_number in ('''

# status_query_end = ')'

# In[4]:



status_query_start = '''select 
            sh.airwaybill_number as 'Airwaybill no',
            sh.order_number as 'Order no',
            sh.product_type as 'Product Type',
            sh.declared_value as 'Declared value',
            sh.collectable_value as 'Collectable Value',
            ' ' as 'Pickup Type',
            IFNULL(DATE_FORMAT(sh.inscan_date,'%d-%b-%Y %H:%i:%s'),'') as 'Pickup Date',
            case when sh.rts_status IN (1) then ls.pincode when sh.reverse_pickup=1 then la.pincode else sh.pincode end as 'Destination Pincode',
            case when sh.reverse_pickup = 1 then rs.pickup_pincode when sh.rts_status = 1 then sh.pincode else p.pincode end as 'Origin Pincode',
            IFNULL(sc.center_name,sc1.center_name) as 'Origin SC',
            case when sh.rts_status IN (1) then l_dst.center_name else l_dst.center_name end as 'Destination SC',
            dest_orig.center_name as 'Manifest Location',
            r_org.region_name as 'Origin Region',
            r_dst.region_name as 'Destn Region',
            s_org.state_name as 'Origin State',
            s_dst.state_name as 'Destn State',
            CONCAT(c.name,' - ' , c.code) as 'Shipper',
            IFNULL(DATE_FORMAT(sh.added_on, '%d-%b-%Y %H:%i'),'') as 'Registered Time',
            ifnull((select distinct 'Yes' from addon_services_shipmentqualitycheck where shipment_id=sh.id),'No') as 'RVP QC',
            ifnull((select group_concat(distinct upper(aos.code) separator ',') from  addon_services_addonservices aos inner join addon_services_shipmentservices aoss on aoss.addonservice_id = aos.id where aoss.shipment_id=sh.id),'N/A') as 'VAS Services',
            sh.pieces as 'No. Of Pieces',
            sh.rts_status as 'RTS Status',
            CASE WHEN (IFNULL(api.transport_type,0) = 0) THEN 'No' ELSE 'Yes' END as 'DG Shipment',
            CASE WHEN (sh.original_dest_id <> sh.service_centre_id and sh.rts_status<>1) THEN concat('Yes (', dest_orig.center_shortcode , ' -> ', l_dst.center_shortcode, ')' ) ELSE 'No' END as 'Misrouted',
            IFNULL((select LEFT(concat(group_concat(case when (rto.status=0) then concat('Yes (', DATE_FORMAT(rto.updated_on,'%d-%b-%Y %H:%i'),')') when (status=1) then concat('Revoked (', DATE_FORMAT(rto.updated_on,'%d-%b-%Y %H:%i'),')') else '' end order by updated_on desc separator '|'),'|'), INSTR(concat(group_concat(case when (rto.status=0) then concat('Yes (', DATE_FORMAT(rto.updated_on,'%d-%b-%Y %H:%i'),')') when (status=1) then concat('Revoked (', DATE_FORMAT(rto.updated_on,'%d-%b-%Y %H:%i'),')') else '' end order by updated_on desc separator '|'),'|'),'|')-1) latest_status  from apiv2_shipmentcancelqueue rto where rto.airwaybill_number = sh.airwaybill_number group by airwaybill_number),'') as 'RTO Lock Status',
            IFNULL((select LEFT(concat(group_concat(case when (ofd.status=0) then concat('Yes (', DATE_FORMAT(ofd.updated_on,'%d-%b-%Y %H:%i'),')') when (ofd.status=1) then concat('Revoked (', DATE_FORMAT(ofd.updated_on,'%d-%b-%Y %H:%i'),')') else '' end order by ofd.updated_on desc separator '|'),'|'), INSTR(concat(group_concat(case when (ofd.status=0) then concat('Yes (', DATE_FORMAT(ofd.updated_on,'%d-%b-%Y %H:%i'),')') when (ofd.status=1) then concat('Revoked (', DATE_FORMAT(ofd.updated_on,'%d-%b-%Y %H:%i'),')') else '' end order by ofd.updated_on desc separator '|'),'|'),'|')-1) latest_status from service_centre_ofdlockqueue ofd where ofd.shipment_id = sh.id group by ofd.shipment_id),'') as 'OFD Lock Status',
            sh.updated_on as 'Status Updated On',
            
            COALESCE((SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM xxxx_xxxx_xxxx_xxxx h WHERE h.shipment_id = sh.id AND  h.status IN (50,4,5,2011,2001,2005,2008) order by id  limit 1),(SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM xxxx_xxxx_xxxx_xxxx h  WHERE h.shipment_id = sh.id AND  h.status IN (50,4,5,2011,2001,2005,2008) order by id  limit 1),(SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM xxxx_xxxx_xxxx_xxxx h  WHERE h.shipment_id = sh.id AND  h.status IN (50,4,5,2011,2001,2005,2008) order by id  limit 1), null) as 'First hub inscan time',
            
            COALESCE((SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM xxxx_xxxx_xxxx_xxxx h  WHERE h.shipment_id = sh.id AND  h.status IN (26,2000) order by id  limit 1),(SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM xxxx_xxxx_xxxx_xxxx h  WHERE h.shipment_id = sh.id AND  h.status IN (26,2000) order by id  limit 1),(SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM xxxx_xxxx_xxxx_xxxx h  WHERE h.shipment_id = sh.id AND  h.status IN (26,2000) order by id  limit 1),null) as 'First hub outscan time',
            
            COALESCE((SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM xxxx_xxxx_xxxx_xxxx h  WHERE h.shipment_id = sh.id AND  h.status IN (50,4,5,2011,2001,2005,2008) order by id desc limit 1),(SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM xxxx_xxxx_xxxx_xxxx h  WHERE h.shipment_id = sh.id AND  h.status IN (50,4,5,2011,2001,2005,2008) order by id desc limit 1),(SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM xxxx_xxxx_xxxx_xxxx h  WHERE h.shipment_id = sh.id AND  h.status IN (50,4,5,2011,2001,2005,2008) order by id desc limit 1),null) as 'Latest hub inscan time',
            
            COALESCE((SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM xxxx_xxxx_xxxx_xxxx h  WHERE h.shipment_id = sh.id AND  h.status IN (26,2000) order by id desc limit 1),(SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM xxxx_xxxx_xxxx_xxxx h  WHERE h.shipment_id = sh.id AND  h.status IN (26,2000) order by id desc limit 1),(SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM xxxx_xxxx_xxxx_xxxx h  WHERE h.shipment_id = sh.id AND  h.status IN (26,2000) order by id desc limit 1),null) as 'Latest hub outscan time',
            
            IFNULL(DATE_FORMAT(ship_latest_status_update_time, '%d-%b-%Y %H:%i:%s'),'') as 'Last Scan',
         
            IFNULL(DATE_FORMAT(ifnull(case when (sm.parent_code in (1224)) then IFNULL((select r.scheduled_delivery_date from track_me_callcentrecomment c inner join track_me_callcentrecommentresolution r on c.id = r.call_centre_comment_id where c.shipments_id = sh.id order by c.date desc limit 1 ),Null) else Null end, (select IFNULL(DATE_FORMAT(IFNULL(STR_TO_DATE(add_info_value,'%Y-%m-%d'),STR_TO_DATE(add_info_value,'%d/%m/%Y')),'%d-%b-%Y'),add_info_value)  from service_centre_additionalinformation where shipment_id = sh.id and add_info_key = 'DATE' LIMIT 1)), '%d-%b-%Y'),'') as 'Scheduled Date',
            IFNULL(last_added_bag_number,'') as 'Bag Number',
            
            @latest_status := CONCAT(IFNULL((select LEFT(concat(group_concat(status order by updated_on desc separator '|'),'|'), INSTR(concat(group_concat(status order by updated_on desc separator '|'),'|'),'|')-1) latest_status  from xxxx_xxxx_xxxx_xxxx h where shipment_id=sh.id and status not in (11,12,16, 20, 75,88) group by shipment_id),''), IFNULL((select LEFT(concat(group_concat(status order by updated_on desc separator '|'),'|'), INSTR(concat(group_concat(status order by updated_on desc separator '|'),'|'),'|')-1) latest_status  from xxxx_xxxx_xxxx_xxxx h where shipment_id=sh.id and status not in (11,12,16, 20, 75,88) group by shipment_id),''),IFNULL((select LEFT(concat(group_concat(status order by updated_on desc separator '|'),'|'), INSTR(concat(group_concat(status order by updated_on desc separator '|'),'|'),'|')-1) latest_status from xxxx_xxxx_xxxx_xxxx h where shipment_id=sh.id and status not in (11,12,16, 20, 75,88) group by shipment_id),'')) as 'Status Number',
            CONCAT( CASE WHEN (@latest_status=8 and sh.rts_status=2) THEN 'Returned' WHEN (IFNULL(ship_current_status_code,'') = '777' and sh.rts_status=2) THEN 'Returned' WHEN (@latest_status in (0,310)) THEN 'Shipment Uploaded' WHEN (@latest_status=1) THEN 'Pickup Complete / Inscan' WHEN (@latest_status=2) THEN 'Inscan completion / Ready for Bagging' WHEN (@latest_status=3) THEN 'Bagging completed' WHEN (@latest_status=4) THEN 'Shipment at HUB' WHEN (@latest_status=5) THEN 'Bagging completed at Hub' WHEN (@latest_status=6) THEN 'Shipment at Delivery Centre' WHEN (@latest_status=7) THEN 'Outscan' WHEN (@latest_status=8) THEN 'Undelivered' WHEN (@latest_status=9) THEN 'Delivered / Closed' WHEN (@latest_status=11) THEN 'Alternate Instruction given' WHEN (@latest_status=12) THEN 'Complaint Registered' WHEN (@latest_status=13) THEN 'Assigned to Run Code' WHEN (@latest_status=14) THEN 'Airport Confirmation Sucessfull, connected to destination via Service Centre' WHEN (@latest_status=15) THEN 'Airport Confirmation Successfull, connected to destination via Hub' WHEN (@latest_status=17) THEN CASE WHEN (IFNULL(sh.ref_airwaybill_number,'') = '') THEN 'Redirection under Same Airwaybill' ELSE concat('Redirection under new airwaybill number ' , IFNULL(sh.ref_airwaybill_number,'')) END WHEN (@latest_status=20) THEN 'Undelivered' WHEN (@latest_status=21) THEN  concat('Tele Calling (', ifnull(sm.code_description,''), ')') WHEN (@latest_status=26) THEN IFNULL(ship_remarks,'Connected from Hub') WHEN (@latest_status=27) THEN IFNULL(ship_remarks,'Connected from SC') WHEN (@latest_status=31) THEN 'Reverse registered' WHEN (@latest_status=32) THEN 'Reverse out for pickup' WHEN (@latest_status=33) THEN 'Reverse not picked up' WHEN (@latest_status=40) THEN 'Mass Update' WHEN (@latest_status=50) THEN 'Inscan At Hub' WHEN (@latest_status=55) THEN 'Bag Inscan At DC' WHEN (@latest_status=77) THEN 'Shipment RTO Lock' WHEN (@latest_status=80) THEN 'RTO Lock Revoked' WHEN (@latest_status=82) THEN 'OFD Lock Applied' WHEN (@latest_status=83) THEN 'OFD Lock Revoked' WHEN (ship_current_status_code=333) THEN 'Lost' WHEN (@latest_status=2000) THEN concat('In-Transit (',ifnull(ship_remarks,''),')') WHEN (@latest_status=2001) THEN 'Bag Debagged at HUB' WHEN (@latest_status=2002) THEN 'Bag Debagged at DC' WHEN (@latest_status=2003) THEN 'Bag Debagged at PPC' WHEN (@latest_status=2004) THEN 'Trip inscan at PPC' WHEN (@latest_status=2005) THEN 'Trip inscan at HUB' WHEN (@latest_status=2006) THEN 'Trip inscan at DC' WHEN (@latest_status=2007) THEN 'Trip Unload at PPC' WHEN (@latest_status=2008) THEN 'Trip Unload at HUB' WHEN (@latest_status=2009) THEN 'Trip Unload at DC' WHEN (@latest_status=2010) THEN 'Trip load / unload complete at PPC' WHEN (@latest_status=2011) THEN 'Trip load / unload complete at HUB' WHEN (@latest_status=2012) THEN 'Trip load / unload complete at DC' ELSE @latest_status END ) as 'Status',
            @latest_reason_code := ifnull(sm.code,'') as 'Reason Code',
            ifnull(CASE WHEN (@latest_reason_code='') THEN '' else (select IFNULL(code_description,'') from ecomm_admin_shipmentstatusmaster where code=@latest_reason_code) END,'') as 'Reason Code Description',
            replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(replace(CONCAT(IFNULL((select LEFT(concat(group_concat(remarks order by updated_on desc separator '|'),'|'), INSTR(concat(group_concat(remarks order by updated_on desc separator '|'),'|'),'|')-1) latest_status  from xxxx_xxxx_xxxx_xxxx h where shipment_id=sh.id  group by shipment_id),''), IFNULL((select LEFT(concat(group_concat(remarks order by updated_on desc separator '|'),'|'), INSTR(concat(group_concat(remarks order by updated_on desc separator '|'),'|'),'|')-1) latest_status  from xxxx_xxxx_xxxx_xxxx h where shipment_id=sh.id  group by shipment_id),''), IFNULL((select LEFT(concat(group_concat(remarks order by updated_on desc separator '|'),'|'), INSTR(concat(group_concat(remarks order by updated_on desc separator '|'),'|'),'|')-1) latest_status  from xxxx_xxxx_xxxx_xxxx h where shipment_id=sh.id  group by shipment_id),'') ),',',''),'/n',''),char(9),''),char(10),''),char(13),''),char(34),''),1,'X'),2,'X'),3,'X'),4,'X'),5,'X'),6,'X'),7,'X'),8,'X'),9,'X'),0,'X') as 'Remarks',
            (select count(distinct case when scdo.status > 0 and sh.product_type in ('eds') then scdo.deliveryoutscan_id when sh.rts_status in (0,2) and sh.product_type not in ('eds') then scdo.deliveryoutscan_id  else null end) from service_centre_doshipment scdo where sh.id = scdo.shipment_id limit 1) as  'No_of_attempts',
            
            ifnull(COALESCE((SELECT lsc.center_shortcode FROM xxxx_xxxx_xxxx_xxxx h  left join location_servicecenter lsc on lsc.id = h.current_sc_id  WHERE h.shipment_id = sh.id  AND h.status IN (1,2,3,4,6,50,55) order by h.id desc limit 1),(SELECT lsc.center_shortcode FROM xxxx_xxxx_xxxx_xxxx h  left join location_servicecenter lsc on lsc.id = h.current_sc_id WHERE h.shipment_id = sh.id  AND h.status IN (1,2,3,4,6,50,55) order by h.id desc limit 1),(SELECT lsc.center_shortcode FROM xxxx_xxxx_xxxx_xxxx h  left join location_servicecenter lsc on lsc.id = h.current_sc_id WHERE h.shipment_id = sh.id AND h.status IN (1,2,3,4,6,50,55) order by h.id desc limit 1), null),CASE WHEN (IFNULL(bag_latest_inscan_hub_time,'1900-01-01') > IFNULL(bag_latest_inscan_dc_time,'1900-01-01')) THEN latest_sc_bag1.center_name ELSE IFNULL(IFNULL(latest_sc1.center_name,latest_sc.center_name),'') END ) as 'Current SC',
            COALESCE((SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM xxxx_xxxx_xxxx_xxxx h  WHERE h.shipment_id = sh.id AND h.current_sc_id = sh.service_centre_id AND h.status IN (55,6) order by id limit 1),(SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM xxxx_xxxx_xxxx_xxxx h  WHERE h.shipment_id = sh.id AND h.current_sc_id = sh.service_centre_id AND h.status IN (55 , 6) order by id limit 1),(SELECT DATE_FORMAT(h.updated_on,'%d-%b-%Y %H:%i:%s') FROM xxxx_xxxx_xxxx_xxxx h  WHERE h.shipment_id = sh.id AND h.current_sc_id = sh.service_centre_id AND h.status IN (55 , 6) order by id limit 1), null) as 'Inscan at DC Date',
            COALESCE((SELECT lsc.center_shortcode FROM xxxx_xxxx_xxxx_xxxx h  left join location_servicecenter lsc on lsc.id = h.current_sc_id  WHERE h.shipment_id = sh.id AND h.current_sc_id = sh.service_centre_id AND h.status IN (55,6) order by h.id limit 1),(SELECT lsc.center_shortcode FROM xxxx_xxxx_xxxx_xxxx h  left join location_servicecenter lsc on lsc.id = h.current_sc_id WHERE h.shipment_id = sh.id AND h.current_sc_id = sh.service_centre_id AND h.status IN (55 , 6) order by h.id limit 1),(SELECT lsc.center_shortcode FROM xxxx_xxxx_xxxx_xxxx h  left join location_servicecenter lsc on lsc.id = h.current_sc_id WHERE h.shipment_id = sh.id AND h.current_sc_id = sh.service_centre_id AND h.status IN (55 , 6) order by h.id limit 1),null) as 'Inscanned DC',
            
            (Select IFNULL(DATE_FORMAT(min(scdo.added_on),'%d-%b-%Y %H:%i:%s'),'') from  service_centre_doshipment scdo inner join service_centre_deliveryoutscan scde on scde.id = scdo.deliveryoutscan_id where sh.id = scdo.shipment_id and (scdo.added_on > sh.inscan_date )) as 'First Attempted Date',
            CASE WHEN (IFNULL(ship_first_status_update_reason_code,'')='') THEN IFNULL(mass_updation_status,'') ELSE IFNULL(ship_first_status_update_reason_code,'') END as 'First Reason Code',
            
            IFNULL(sm_first.code_description,IFNULL(mass_updation_status,'')) as 'First Reason',
            
            CASE WHEN (IFNULL(ship_latest_status_update_reason_code,'')='') THEN IFNULL(mass_updation_status,'') ELSE IFNULL(ship_latest_status_update_reason_code,'') END as 'Last Reason Code',
            IFNULL(sm_last.code_description,IFNULL(mass_updation_status,'')) as 'Last Reason',
            IFNULL(sh.ref_airwaybill_number,'') as 'Ref AWB',
            fwd.inscan_date as 'Parent Inscan Date',
            
            fwd.added_on as 'Parent Added Date',
            (Select IFNULL(DATE_FORMAT(max(scdo.added_on),'%d-%b-%Y %H:%i:%s'),'') from  service_centre_doshipment scdo inner join service_centre_deliveryoutscan scde on scde.id = scdo.deliveryoutscan_id where sh.id = scdo.shipment_id and (scdo.added_on > sh.inscan_date )) as 'Last Outscan time',
            (Select IFNULL(DATE_FORMAT(min(scdo.added_on),'%d-%b-%Y %H:%i:%s'),'') from  service_centre_doshipment scdo inner join service_centre_deliveryoutscan scde on scde.id = scdo.deliveryoutscan_id where sh.id = scdo.shipment_id and sh.reverse_pickup = 1 and (scdo.added_on <= sh.inscan_date or sh.inscan_date is null)) as 'RVP_first_pickup_attempt',
            (Select IFNULL(DATE_FORMAT(max(scdo.added_on),'%d-%b-%Y %H:%i:%s'),'') from  service_centre_doshipment scdo inner join service_centre_deliveryoutscan scde on scde.id = scdo.deliveryoutscan_id where sh.id = scdo.shipment_id and sh.reverse_pickup = 1 and (scdo.added_on <= sh.inscan_date or sh.inscan_date is null)) as 'RVP_latest_pickup_attempt',
            
            CONCAT(IFNULL((select ifnull(DATE_FORMAT(max(h1.updated_on), '%d-%b-%Y %H:%i'),'') latest_status from xxxx_xxxx_xxxx_xxxx  h1 where shipment_id=sh.id and status not in (11,12,16, 20, 75,77,80) and reason_code_id not in(127,84,85,86,87,88,77,116,117,228,229,230,232,232,116,117)),'') ,IFNULL((select ifnull(DATE_FORMAT(max(h1.updated_on), '%d-%b-%Y %H:%i'),'') latest_status from xxxx_xxxx_xxxx_xxxx  h1 where shipment_id=sh.id and status not in (11,12,16, 20, 75,77,80) and reason_code_id not in(127,84,85,86,87,88,77,116,117,228,229,230,232,232,116,117)),'') ,IFNULL((select ifnull(DATE_FORMAT(max(h1.updated_on), '%d-%b-%Y %H:%i'),'') latest_status from xxxx_xxxx_xxxx_xxxx  h1 where shipment_id=sh.id and status not in (11,12,16, 20, 75,77,80) and reason_code_id not in(127,84,85,86,87,88,77,116,117,228,229,230,232,232,116,117)),'')) as 'Last Updated date',
            
            ifnull((select date(sa.add_info_value) from service_centre_additionalinformation sa where sh.id = sa.shipment_id and sa.add_info_key='ORIGINAL_EDD'  limit 1),'') as 'Actual_EDD',
            
            date(sh.expected_dod) as 'Updated_EDD',
            @latest_Outscanned_time := (select o.added_on from service_centre_deliveryoutscan o inner join service_centre_doshipment ds on o.id = ds.deliveryoutscan_id where ds.shipment_id = sh.id and o.status>0 order by o.added_on desc limit 1) as 'latest Outscanned time',(select concat(ifnull(ae.firstname,''),' ',ifnull(ae.lastname,''),'- (',ae.employee_code,')') from service_centre_deliveryoutscan o inner join service_centre_doshipment ds on o.id = ds.deliveryoutscan_id left join authentication_employeemaster ae on ae.id = o.employee_code_id where ds.shipment_id = sh.id and o.status>0 and o.added_on = @latest_Outscanned_time order by o.added_on desc limit 1) as 'latest Outscanned by',
            
@latest_status := 
    IFNULL((select LEFT(concat(group_concat(status order by updated_on desc separator '|'),'|'), 
    INSTR(concat(group_concat(status order by updated_on desc separator '|'),'|'),'|')-1) latest_status 
    from xxxx_xxxx_xxxx_xxxx where shipment_id=sh.id and status not in (11,12,16, 20, 75,88) group by shipment_id),'')
    as 'Status_No',
    
@STATUS:=   CONCAT( CASE WHEN (@latest_status=8 and sh.rts_status=2) THEN 'Returned' 
                                WHEN (IFNULL(ship_current_status_code,'') = '777' and sh.rts_status=2) THEN 'Returned' 
                                WHEN (@latest_status in (0,310)) THEN 'Shipment Uploaded' 
                                WHEN (@latest_status=1) THEN 'Pickup Complete / Inscan' WHEN (@latest_status=2) THEN 'Inscan completion / Ready for Bagging' 
                                WHEN (@latest_status=3) THEN 'Bagging completed' WHEN (@latest_status=4) THEN 'Shipment at HUB' 
                                WHEN (@latest_status=5) THEN 'Bagging completed at Hub' WHEN (@latest_status=6) THEN 'Shipment at Delivery Centre' 
                                WHEN (@latest_status=7) THEN 'Outscan' WHEN (@latest_status=8) THEN 'Undelivered' WHEN (@latest_status=9) THEN 'Delivered / Closed'
                                WHEN (@latest_status=11) THEN 'Alternate Instruction given' WHEN (@latest_status=12) THEN 'Complaint Registered' 
                                WHEN (@latest_status=13) THEN 'Assigned to Run Code' 
                                WHEN (@latest_status=14) THEN 'Airport Confirmation Sucessfull, connected to destination via Service Centre' 
                                WHEN (@latest_status=15) THEN 'Airport Confirmation Successfull, connected to destination via Hub' 
                                WHEN (@latest_status=17) THEN CASE WHEN (IFNULL(sh.ref_airwaybill_number,'') = '') THEN 'Redirection under Same Airwaybill' ELSE concat('Redirection under new airwaybill number ' , IFNULL(sh.ref_airwaybill_number,'')) END WHEN (@latest_status=20) THEN 'Undelivered' 
                                WHEN (@latest_status=21) THEN  'Tele Calling' 
                                WHEN (@latest_status=26) THEN IFNULL(ship_remarks,'Connected from Hub') WHEN (@latest_status=27) THEN IFNULL(ship_remarks,'Connected from SC') 
                                WHEN (@latest_status=31) THEN 'Reverse registered' WHEN (@latest_status=32) THEN 'Reverse out for pickup' 
                                WHEN (@latest_status=33) THEN 'Reverse not picked up' WHEN (@latest_status=40) THEN 'Mass Update' 
                                WHEN (@latest_status=50) THEN 'Inscan At Hub' WHEN (@latest_status=55) THEN 'Bag Inscan At DC' 
                                WHEN (@latest_status=77) THEN 'Shipment RTO Lock' WHEN (@latest_status=80) THEN 'RTO Lock Revoked' 
        WHEN (@latest_status=82) THEN 'OFD Lock Applied' WHEN (@latest_status=83) THEN 'OFD Lock Revoked' 
        WHEN (ship_current_status_code=333) THEN 'Lost' WHEN (@latest_status=2000) THEN concat('In-Transit (',ifnull(ship_remarks,''),')') 
        WHEN (@latest_status=2001) THEN 'Bag Debagged at HUB' WHEN (@latest_status=2002) THEN 'Bag Debagged at DC' 
        WHEN (@latest_status=2003) THEN 'Bag Debagged at PPC' WHEN (@latest_status=2004) THEN 'Trip inscan at PPC' 
        WHEN (@latest_status=2005) THEN 'Trip inscan at HUB' WHEN (@latest_status=2006) THEN 'Trip inscan at DC' 
        WHEN (@latest_status=2007) THEN 'Trip Unload at PPC' WHEN (@latest_status=2008) THEN 'Trip Unload at HUB' 
        WHEN (@latest_status=2009) THEN 'Trip Unload at DC' WHEN (@latest_status=2010) THEN 'Trip load / unload complete at PPC' 
        WHEN (@latest_status=2011) THEN 'Trip load / unload complete at HUB' WHEN (@latest_status=2012) THEN 'Trip load / unload complete at DC' 
        ELSE @latest_status END ) 
    as 'Status_details',
@rcode := (select  sm.code from xxxx_xxxx_xxxx_xxxx h inner join ecomm_admin_shipmentstatusmaster sm on h.reason_code_id = sm.id where  h.shipment_id = sh.id order by h.updated_on desc limit 1) 'Reason_Code',
   
@rcode_desc := (select  sm.code_description from xxxx_xxxx_xxxx_xxxx h inner join ecomm_admin_shipmentstatusmaster sm on h.reason_code_id = sm.id where  h.shipment_id = sh.id order by h.updated_on desc limit 1) 'Reason_Code_Description',
   
@rcode_udpate_time:= (select  h.updated_on from xxxx_xxxx_xxxx_xxxx h inner join ecomm_admin_shipmentstatusmaster sm on h.reason_code_id = sm.id where  h.shipment_id = sh.id order by h.updated_on desc limit 1) 'Reason_Code_update_time',
@rcode_udpate_location:= (select  lsc.center_shortcode from xxxx_xxxx_xxxx_xxxx h inner join ecomm_admin_shipmentstatusmaster sm on h.reason_code_id = sm.id left join location_servicecenter lsc on lsc.id=h.current_sc_id where  h.shipment_id = sh.id order by h.updated_on desc limit 1)  'Reason_Code_Location',
@Current_sc:=COALESCE((SELECT lsc.center_shortcode FROM xxxx_xxxx_xxxx_xxxx h left join location_servicecenter lsc on lsc.id = h.current_sc_id  WHERE h.shipment_id = sh.id  AND h.status IN (0,1,2,3,4,5,6,7,8,17,26,27,40,50,55,77,82,83,1220,1230,1340,1350,1400,1410,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2070,2071,2072) and h.current_sc_id not in (135,4080,97,3956) order by h.updated_on desc limit 1),
CASE WHEN (IFNULL(bag_latest_inscan_hub_time,'1900-01-01') > IFNULL(bag_latest_inscan_dc_time,'1900-01-01')) THEN latest_sc_bag1.center_shortcode ELSE IFNULL(IFNULL(latest_sc1.center_shortcode,latest_sc.center_shortcode),'') END ) 'latest_location',

(SELECT h.updated_on FROM xxxx_xxxx_xxxx_xxxx h  left join location_servicecenter lsc on lsc.id = h.current_sc_id WHERE h.shipment_id = sh.id and @Current_sc = lsc.center_shortcode and h.status not in (16) order by h.updated_on desc limit 1 ) 'last_updated_time',
(SELECT h.updated_on FROM xxxx_xxxx_xxxx_xxxx h 
WHERE h.shipment_id = sh.id AND h.current_sc_id = sh.service_centre_id AND h.status IN (55 , 6) order by id limit 1)
'Inscan at DC Date',


(SELECT lsc.center_shortcode FROM xxxx_xxxx_xxxx_xxxx  h 
left join location_servicecenter lsc on lsc.id = h.current_sc_id  WHERE h.shipment_id = sh.id 
AND h.current_sc_id = sh.service_centre_id AND h.status IN (55 , 6) order by h.updated_on limit 1) 'dc_inscan',
sh.updated_on 'last_update',
case when (sh.original_dest_id <>sh.service_centre_id) then 'yes' else 'no' end 'misroute',
(select count(distinct(scdo.deliveryoutscan_id)) from service_centre_doshipment scdo 
where sh.id = scdo.shipment_id and sh.inscan_date<scdo.added_on limit 1) as  'No_of_attempts',
(Select IFNULL(DATE_FORMAT(min(scdo.added_on),'%d-%b-%Y %H:%i:%s'),'') 
from  service_centre_doshipment scdo inner join service_centre_deliveryoutscan scde on
 scde.id = scdo.deliveryoutscan_id where sh.id = scdo.shipment_id and (scdo.added_on > sh.inscan_date )) as 'First Attempted Date',
(Select IFNULL(DATE_FORMAT(max(scdo.added_on),'%d-%b-%Y %H:%i:%s'),'') 
from  service_centre_doshipment scdo inner join service_centre_deliveryoutscan scde on
 scde.id = scdo.deliveryoutscan_id where sh.id = scdo.shipment_id and (scdo.added_on > sh.inscan_date )) as 'Last Attempted Date'
,
c.name 'Shipper_name'



from service_centre_shipment sh
             inner join customer_customer c on c.id = sh.shipper_id
             left join reports_shipmentbaghistory s on sh.airwaybill_number=s.awb
             left join service_centre_shipment fwd on fwd.airwaybill_number = sh.ref_airwaybill_number
             left join location_servicecenter l_dst on sh.service_centre_id = l_dst.id
             left join location_city c_dst on c_dst.id = l_dst.city_id
             left join location_state s_dst on s_dst.id = c_dst.state_id
             left join location_region r_dst on r_dst.id = c_dst.region_id
             left join ecomm_admin_shipmentstatusmaster sm on sm.id = sh.reason_code_id
             left join pickup_pickupregistration p on sh.pickup_id=p.id
             left join location_servicecenter sc on p.service_centre_id = sc.id
             left join location_city c_org on c_org.id = sc.city_id
             left join location_state s_org on s_org.id = c_org.state_id
             left join location_region r_org on r_org.id = c_org.region_id
             left join location_servicecenter sc1 on s.ship_added_on_sc = sc1.center_shortcode 
             left join ecomm_admin_shipmentstatusmaster sm_last on sm_last.code = ifnull(s.ship_latest_status_update_reason_code,mass_updation_status) 
             left join location_servicecenter latest_sc1 on latest_sc1.id=sh.current_sc_id
             left join location_servicecenter dest_orig on dest_orig.id=sh.original_dest_id
             left join api_apishipments api on api.shipment_id = sh.id
             left join location_servicecenter latest_sc_bag on latest_sc_bag.center_shortcode=bag_latest_inscan_dc_sc
             left join location_servicecenter latest_sc_bag1 on latest_sc_bag1.center_shortcode=bag_latest_inscan_hub_sc
             left join ecomm_admin_shipmentstatusmaster sm_first on sm_first.code = ifnull(s.ship_first_status_update_reason_code,mass_updation_status)
             left join location_servicecenter latest_sc on latest_sc.center_shortcode=ship_latest_inscan_dc_sc
             left join customer_shipper cs on cs.id = p.return_subcustomer_code_id
             left join location_address ls on ls.id = cs.address_id
             left join location_pincode rp on rp.pincode = ls.pincode
             left join location_servicecenter rc on rc.id =rp.service_center_id
             left join service_centre_reverseshipment rs on sh.airwaybill_number = rs.airwaybill_number
             left join pickup_pickupregistration p1 on p1.id = rs.pickup_id
             left join customer_shipper cs1 on cs1.id = p1.subcustomer_code_id
             left join location_address la on la.id = cs1.address_id
            

   
   where sh.airwaybill_number in (
'''
status_query_end = ')'


# In[5]:



print('\nWorking of status Query\n')


# In[11]:


history_table = list(data.Applicable_History_Table.unique())


# In[12]:


try:
    del data_status
except:
    pass

try:
    data_list=[]
    for x in history_table:
        print('Working in 50K batches History Table Wise: Considering %s'%x)
        work_data = data[data.Applicable_History_Table==x]
        work_data = work_data.reset_index().drop(columns='index')
        a,b = value_interpreter(len(work_data[work_data.columns[0]]),50000)
        for i,j in enumerate(b):
            if ((i+1)==len(b)):
                pass
            else:
                print('Working Range : ',str(j),'-',str(b[i+1]))
                query_start,query_end = query_history(status_query_start,status_query_end,x,'xxxx_xxxx_xxxx_xxxx')
                query_data = query_start+(',').join(work_data[work_data.columns[0]][j:b[i+1]].apply(lambda x:str(int(x))))+query_end
                conn = connections()
                temp_data = pd.read_sql(query_data,conn)
                print('Saving_data of Batch: %d of Total Batch %d\n'%(i+1,len(b)-1))
                data_list.append(temp_data)
                #temp_data.to_csv(fd_path+str(i)+'_fd_1_.csv',index=False)
    for i in data_list:
        try:
            data_status = data_status.append(i)
        except:
            data_status = i

    print('Status Data imported from SQL')
    data_status = data_status.reset_index().drop(columns='index')
except:
    print('Error in status Query time')
    data_list=[]
    for x in history_table:
        print('Working in 50K batches History Table Wise: Considering %s'%x)
        work_data = data[data.Applicable_History_Table==x]
        work_data = work_data.reset_index().drop(columns='index')
        a,b = value_interpreter(len(work_data[work_data.columns[0]]),50000)
        for i,j in enumerate(b):
            if ((i+1)==len(b)):
                pass
            else:
                print('Working Range : ',str(j),'-',str(b[i+1]))
                query_start,query_end = query_history(status_query_start,status_query_end,x,'xxxx_xxxx_xxxx_xxxx')
                query_data = query_start+(',').join(work_data[work_data.columns[0]][j:b[i+1]].apply(lambda x:str(int(x))))+query_end
                conn = connections()
                temp_data = pd.read_sql(query_data,conn)
                print('Saving_data of Batch: %d of Total Batch %d\n'%(i+1,len(b)-1))
                data_list.append(temp_data)
                #temp_data.to_csv(fd_path+str(i)+'_fd_1_.csv',index=False)
    for i in data_list:
        try:
            data_status = data_status.append(i)
        except:
            data_status = i

    print('Status Data imported from SQL')
    data_status = data_status.reset_index().drop(columns='index')



data_status.to_csv('output_data1.csv',index=False)


# In[ ]:





# In[ ]:



AWB Input Value: 13
Working in 50 K batches
Total AWB: 13
Working Range :  0 - 13
Saving_data of Batch: 1 of Total Batch 1

Raw Data imported from SQL

Biforcation
FORWARD    13
Name: Shipment_type, dtype: int64

History Table Applicable
service_centre_shipmenthistory_2021_10    13
Name: Applicable_History_Table, dtype: int64

Working of status Query

Working in 50K batches History Table Wise: Considering service_centre_shipmenthistory_2021_10
Working Range :  0 - 13
Error in status Query time
Working in 50K batches History Table Wise: Considering service_centre_shipmenthistory_2021_10
Working Range :  0 - 13


DatabaseError: Execution failed on sql 'select 
            sh.airwaybill_number as 'Airwaybill no',
            sh.order_number as 'Order no',
            sh.product_type as 'Product Type',
            sh.declared_value as 'Declared value',
            sh.collectable_value as 'Collectable Value',
            ' ' as 'Pickup Type',
            IFNULL(DATE_FORMAT(sh.inscan_date,'%d-%b-%Y %H:%i:%s'),'') as 'Pickup Date',
            case when sh.rts_status IN (1) then ls.pincode when sh.reverse_pickup=1 then la.pincode else sh.pincode end as 'Destination Pincode',
            case when sh.reverse_pickup = 1 then rs.pickup_pincode when sh.rts_status = 1 then sh.pincode else p.pincode end as 'Origin Pincode',
            IFNULL(sc.center_name,sc1.center_name) as 'Origin SC',
            case when sh.rts_status IN (1) then l_dst.center_name else l_dst.center_name end as 'Destination SC',
            dest_orig.center_name as 'Manifest Location',
            r_org.region_name as 'Origin Region